In [137]:
include Math

require 'numo/narray'

def log(x)
  Numo::NMath.log(x)
end

def exp(x)
  Numo::NMath.exp(x)
end

def sin(x)
  Numo::NMath.sin(x)
end

def cos(x)
  Numo::NMath.cos(x)
end

def sqrt(x)
  Numo::NMath.sqrt(x)
end

:sqrt

# 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [6]:
pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
sen = (pos_tagged + neg_tagged).shuffle
counter = {"+1" => 0, "-1" => 0}

out = File.open("../data/sentiment.txt", "w")
sen.each do |line|
  out.puts line
end
out.close

open('../data/sentiment.txt').each_line do |line|
  counter[line[0..1]] += 1
end

counter

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [7]:
word_freq = Hash.new { |h, k| h[k] = 0 }

open('../data/sentiment.txt').each_line do |line|
  line[2..-1].chomp.split.each do |word|
    word_freq[word] += 1
  end
end

# 出現頻度が高い単語上位100語をストップワードにした
# stop_list = word_freq.sort {|(_, v1), (_, v2)| v2 <=> v1}[0..100].map {|arr| arr[0]}
# stop_list += word_freq.select {|k, v| v < 5}.map {|arr| arr[0]}

stop_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def is_stopped?(word, stop_list)
  stop_list.include? word
end

# goodとかがストップワードになっちゃった...
puts is_stopped?("lion", stop_list)
puts is_stopped?("does", stop_list)
puts is_stopped?("good", stop_list)

false
true
false


# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．



In [69]:
all_words = []
tf_vecs = []
labels = []

File.foreach('../data/sentiment.txt').with_index do |line, index|  
  tf_vec = Hash.new { |h, k| h[k] = 0 }  
  words = line[3..-1].chomp.split.reject {|word| is_stopped?(word, stop_list) }
  all_words += words
  
  words.each do |word|
    tf_vec[word] += 1
  end
  
  labels << line[0..1]
  tf_vecs << tf_vec  
end

all_words.uniq!
word_index = {}

all_words.each_with_index do |word, index|
  word_index.store(word, index)
end

x = Numo::Float64.zeros(tf_vecs.size, all_words.size)
y = Numo::Float64.zeros(tf_vecs.size, 1)
w = Numo::Float64.new(all_words.size, 1).rand

labels.each_with_index do |label, i|
  y[i] = label == '+1' ? 1.0 : 0.0 
end

tf_vecs.each_with_index do |hash, i|
  hash.each do |word, count|
    x[i, word_index[word]] = count
  end
end

1

1

# 73. 学習
72で抽出した素性を用いて、
ロジスティック回帰モデルを学習せよ．

\begin{align}
J(\theta) = - y log(h_{\theta}(x))  - (1 - y) log(1 - h_{ \theta }(x)) \\
h_{\theta} (x) = \frac{1}{e^{-{\theta}^T x}} \\
\frac{\partial J(\theta)}{\partial x} = x^{T} (h_{\theta}(x) - y)
\end{align}

In [227]:
def cost(w, x, y)
  (1.0 / y.size) * (- y * Numo::NMath.log(h(w, x)) - (1-y) * Numo::NMath.log(1 - h(w, x))).sum
end
  

def h(w, x)
  z = x.dot(w)
  1.0/ (1 + Numo::NMath.exp(-z))
end


def predict(w, x, y, threshold=0.5)
  h(w, x) > threshold
end


def grad(w, x, y)
  x.transpose.dup.dot(h(w, x) - y)
end


def check(w, w_tmp, x, y)
  (cost(w, x, y) - cost(w_tmp, x, y)).abs
end


def update(w, x, y, eta=0.01)
  w - eta * grad(w, x, y)
end


def train(x, y)
  a = Numo::Float64.new(x.shape[1], 1).rand
  b = Numo::Float64.new(x.shape[1], 1).rand
  w = sqrt(-2 * log(a)) * cos(2 * PI * b)
  
  100.times do |t|
    w_tmp = w
    w = update(w, x, y, 0.01)
    
    puts "iter: #{t}, cost: #{cost(w, x, y)}" if t % 5 == 0
    return w if check(w, w_tmp, x, y) < 0.02
  end
  
  return w
end

:train

In [229]:
w = train(x, y)

iter: 0, cost: 43.86496574765914
iter: 5, cost: Infinity
iter: 10, cost: 43.86937091489254
iter: 15, cost: Infinity
iter: 20, cost: 43.87265989883706
iter: 25, cost: Infinity
iter: 30, cost: 43.87476739441467
iter: 35, cost: Infinity
iter: 40, cost: 43.876010330012065
iter: 45, cost: Infinity
iter: 50, cost: 43.876804034570355
iter: 55, cost: Infinity
iter: 60, cost: 43.87748179935985
iter: 65, cost: Infinity
iter: 70, cost: 43.878207420979216
iter: 75, cost: Infinity
iter: 80, cost: 43.87902805471162
iter: 85, cost: Infinity
iter: 90, cost: 43.87993590575979
iter: 95, cost: Infinity


Numo::DFloat#shape=[13940,1]
[[-0.2375], 
 [0.288515], 
 [1.0721], 
 [0.470308], 
 [-0.849865], 
 [0.124201], 
 [-0.834514], 
 [0.689164], 
 [-0.380535], 
 [19.0564], 
 [0.493201], 
 [0.938994], 
 [0.164922], 
 [1.14474], 
 [0.663828], 
 [0.357446], 
 [-0.153245], 
 [30.4003], 
 [2.49401], 
 [-2.06346], 
 ...

In [230]:
h(w, x)

Numo::DFloat#shape=[10660,1]
[[1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 [1], 
 ...

# ... (;-;)